# Mutagenesis Example in Python

Following example is the same as the julia version of the 
Following example demonstrates learning to predict the mutagenicity on Salmonella typhimurium using the [JuliaPy/pyjulia](https://github.com/JuliaPy/pyjulia), and demonstrates how can the JsonGrinder.jl be used entirely from python.

We start by installing the pyjulia and use it to install the JsonGrinder and few other packages we need for the example.

In [1]:
!pip install julia

In [3]:
import julia

In [4]:
julia.install()

[ Info: Julia version info


Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  uname: Linux 5.10.16.3-microsoft-standard-WSL2 #1 SMP Fri Apr 2 22:23:49 UTC 2021 x86_64 x86_64
  CPU: Intel(R) Core(TM) i7-9850H CPU @ 2.60GHz: 
              speed         user         nice          sys         idle          irq
       #1  2592 MHz      10271 s          0 s       6263 s      77778 s          0 s
       #2  2592 MHz      11955 s          0 s       7993 s      77455 s          0 s
       
  Memory: 1.9366111755371094 GB (866.65625 MB free)
  Uptime: 10331.86 sec
  Load Avg:  0.22  14.99  31.88
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)
Environment:
  JULIA_DEPOT_PATH = /opt/julia
  JULIA_PKGDIR = /opt/julia
  JULIA_VERSION = 1.7.2
  PATH = /opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
  HOME = /home/jovyan
  XDG_CACHE_HOME = /home/jovyan/.cache/
  JULIA_DEPOT_PATH = /opt/julia
  TERM = xterm-colo

[ Info: Julia executable: /opt/julia-1.7.2/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /opt/conda/bin/python
│     libpython: /opt/conda/lib/libpython3.9.so.1.0
│ Python:
│     python: /opt/conda/bin/python
└     libpython: 


In [5]:
from julia import Julia
Julia(compiled_modules=False)
#Julia(runtime='C:\\Users\\racinsky\\AppData\\Local\\Programs\\Julia-1.7.2\\bin\\julia.exe')

/opt/conda/lib/python3.9/site-packages/julia/core.py:703: FutureWarning: Accessing `Julia().<name>` to obtain Julia objects is deprecated.  Use `from julia import Main; Main.<name>` or `jl = Julia(); jl.eval('<name>')`.
  warnings.warn(


We check that the correct version of Julia was installed. We need julia 1.6 or newer.

In [6]:
from julia import Base
Base.VERSION

<PyCall.jlwrap 1.7.2>

We install the packages using the julia package manager called from python

Executing the following line may take few minutes, because the package manager needs to resolve proper versions.

In [7]:
from julia import Pkg
Pkg.add("MLDatasets")
Pkg.add("Flux")
Pkg.add("Mill")
Pkg.add("MLDataPattern")
Pkg.add("Statistics")
Pkg.add(name="JsonGrinder", rev="master")


    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`
    Updating git-repo `https://github.com/CTUAvastLab/JsonGrinder.jl.git`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
 

Here we include libraries all necessary libraries

Executing the following cell may take few minutes to finish, which is expected, because there is lots of code being precompiled.

In [45]:
from julia import JsonGrinder, MLDatasets, Flux, Mill, MLDataPattern, Statistics

Here we load all samples.

In order to download datasets automatically from jupyter notebook, make sure the environment variable `DATADEPS_ALWAYS_ACCEPT` is present and set to true. E.g. using this if you run the notebook from docker compose
```
services:
  ...
    environment:
      - DATADEPS_ALWAYS_ACCEPT=true
```


In [9]:
train_x, train_y = MLDatasets.Mutagenesis.traindata();
test_x, test_y = MLDatasets.Mutagenesis.testdata();

We define some basic parameters for the construction and training of the neural network. Minibatch size is self-explanatory, iterations is number of iterations of gradient descent Neurons is number of neurons in hidden layers for each version of part of the neural network.

In [10]:
minibatchsize = 100
iterations = 5_000
neurons = 20

We create the schema of the training data, which is the first important step in using the JsonGrinder. This computes both the structure (also known as JSON schema) and histogram of occurrences of individual values in the training data.

In [11]:
sch = JsonGrinder.schema(train_x)
sch

<PyCall.jlwrap DictEntry>

Then we use it to create the extractor converting jsons to Mill structures. The suggestextractor is executed below with default setting, but it allows you heavy customization. We also prepare list of classes. This classification problem is two-class, but we want to infer it from labels.

In [12]:
extractor = JsonGrinder.suggestextractor(sch)
labelnames = Base.unique(train_y)

# Create the model
We create the model reflecting structure of the data

In [13]:
model = Mill.reflectinmodel(sch, extractor,
    lambda input_dim: Flux.Dense(input_dim, neurons, Flux.relu),
    lambda input_dim: Mill.SegmentedMeanMax(input_dim),
    fsm = {"": lambda input_dim: Flux.Dense(input_dim, Base.length(labelnames))},
)
model

<PyCall.jlwrap ProductModel ↦ ArrayModel(Dense(100, 2))>

We convert jsons to mill data samples and prepare list of classes. This classification problem is two-class, but we want to infer it from labels. The extractor is callable, so we can pass it vector of samples to obtain vector of structures with extracted features.

In [14]:
train_data = list(map(extractor, train_x))
test_data = list(map(extractor, test_x))
test_data

[<PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.jlwrap ProductNode>,
 <PyCall.j

# Train the model
Then, we define few handy functions and a loss function, which is categorical crossentropy in our case.

The loss function needs to be defined in julia, not in python, unfortunately, because of the https://github.com/JuliaPy/pyjulia/issues/491 bug. This is the only code which needs to be ran from julia, and this is the most obscure part of the notebook, which passes label names and model from python to julia so the gradient calculation works properly.

In [41]:
def inference(x): return model(x).data
def accuracy(x,y): return Statistics.mean(labelnames[Flux.onecold(inference(x))-1] == y)
import julia.Main as jl
def loss_factory(model, labelnames, loss_name):
    jl.model__ = model
    jl.labelnames__ = labelnames
    return jl.eval(f"(x, y) -> {loss_name}(model__(x).data, Flux.onehotbatch(y, labelnames__))")


And we can add a callback which will be printing train and test accuracy during the training and then we can start trining

In [37]:
def cb():
    train_acc = accuracy(train_data, train_y)
    test_acc = accuracy(test_data, test_y)
    print(f"accuracy: {train_acc=}, {test_acc=}")

Lastly we turn our training data to minibatches, and we can start training

In [20]:
minibatches = MLDataPattern.RandomBatches((train_data, train_y), size = minibatchsize, count = iterations)

Here we create the actual loss function and store it in python variable.

In [29]:
my_loss = loss_factory(model, labelnames, "Flux.logitcrossentropy")
my_loss

<PyCall.jlwrap #18>

And now we can finally start training.

In [43]:
Flux.Optimise.train_b(my_loss, Flux.params(model), minibatches, Flux.ADAM(), cb = Flux.throttle(cb, 2))

accuracy: train_acc=0.88, test_acc=0.8409090909090909
accuracy: train_acc=0.92, test_acc=0.8409090909090909
accuracy: train_acc=0.97, test_acc=0.7954545454545454
accuracy: train_acc=0.98, test_acc=0.75
accuracy: train_acc=0.99, test_acc=0.7954545454545454
accuracy: train_acc=1.0, test_acc=0.8409090909090909
accuracy: train_acc=1.0, test_acc=0.7727272727272727
accuracy: train_acc=1.0, test_acc=0.7727272727272727
accuracy: train_acc=1.0, test_acc=0.8181818181818182
accuracy: train_acc=1.0, test_acc=0.75
accuracy: train_acc=1.0, test_acc=0.7727272727272727
accuracy: train_acc=1.0, test_acc=0.7727272727272727
accuracy: train_acc=1.0, test_acc=0.8181818181818182
accuracy: train_acc=1.0, test_acc=0.75
accuracy: train_acc=1.0, test_acc=0.8181818181818182
accuracy: train_acc=1.0, test_acc=0.7727272727272727
accuracy: train_acc=1.0, test_acc=0.7954545454545454
accuracy: train_acc=1.0, test_acc=0.7727272727272727
accuracy: train_acc=1.0, test_acc=0.8181818181818182
accuracy: train_acc=1.0, test_

We can see the accuracy rising and obtaining over 99% on training set quite quickly, and on test set we get over 79%.

# Classify test set
The Last part is inference on test data.

In [55]:
probs = Flux.softmax(model(test_data).data)
o = Flux.onecold(probs) - 1
pred_classes = labelnames[o]
Statistics.mean(pred_classes == test_y)

0.7954545454545454

`pred_classes` contains the predictions for our test set. we see the accuracy is around 75% on test set predicted classes for test set

In [57]:
pred_classes

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
      dtype=int64)

Ground truth classes for test set

In [58]:
test_y

array([1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
      dtype=int64)

probabilities for test set

In [59]:
probs

array([[9.99987364e-01, 9.97891963e-01, 8.30009487e-03, 9.97200375e-04,
        9.93917823e-01, 9.97030973e-01, 9.94567633e-01, 3.73403668e-01,
        9.96551633e-01, 9.99986172e-01, 5.99438548e-01, 7.75109977e-02,
        9.73641872e-01, 9.98892844e-01, 6.51718020e-01, 5.34612298e-01,
        9.99948621e-01, 6.08506978e-01, 6.43978885e-04, 3.65056068e-01,
        9.76465583e-01, 1.30008530e-05, 9.90937948e-01, 9.99985695e-01,
        9.02300358e-01, 9.99958515e-01, 6.98983550e-01, 9.99988675e-01,
        1.00000000e+00, 9.99984026e-01, 9.95685101e-01, 2.45655611e-01,
        1.75017258e-03, 2.73398310e-02, 9.99971986e-01, 9.92810369e-01,
        9.99029279e-01, 9.99971271e-01, 9.99995947e-01, 9.97915804e-01,
        9.99993920e-01, 4.54691052e-01, 9.95735168e-01, 9.99857545e-01],
       [1.26657187e-05, 2.10803282e-03, 9.91699874e-01, 9.99002874e-01,
        6.08217902e-03, 2.96906126e-03, 5.43241017e-03, 6.26596332e-01,
        3.44841368e-03, 1.38119785e-05, 4.00561422e-01, 9.22489

We can look at individual samples. For instance, some sample from test set is

In [63]:
Mill.printtree(test_data[2])

ProductNode  # 1 obs, 104 bytes
  ├─── lumo: ArrayNode(99×1 OneHotArray with Bool elements)  # 1 obs, 60 bytes
  ├─── inda: ArrayNode(2×1 OneHotArray with Bool elements)  # 1 obs, 60 bytes
  ├─── logp: ArrayNode(63×1 OneHotArray with Bool elements)  # 1 obs, 60 bytes
  ├─── ind1: ArrayNode(3×1 OneHotArray with Bool elements)  # 1 obs, 60 bytes
  ╰── atoms: BagNode  # 1 obs, 136 bytes
               ╰── ProductNode  # 25 obs, 64 bytes
                     ├──── element: ArrayNode(7×25 OneHotArray with Bool elements)  # 25 obs, 156 bytes
                     ├────── bonds: BagNode  # 25 obs, 488 bytes
                     │                ╰── ProductNode  # 54 obs, 32 bytes
                     │                      ├──── element: ArrayNode(7×54 OneHotArray with Bool elements)  # 54 obs, 272 bytes
                     │                      ├── bond_type: ArrayNode(4×54 OneHotArray with Bool elements)  # 54 obs, 272 bytes
                     │                      ├───── charge: ArrayN

and the corresponding classification is

In [69]:
pred_classes[2]

0

If you want to see the probability distribution, it can be obtained by applying softmax to the output of the network.

In [68]:
Flux.softmax(model(test_data[2]).data)

array([[0.00830009],
       [0.9916999 ]], dtype=float32)

so we can see that the probability that given sample is `mutagenetic` is almost 1.